
# NeuralForecast 分析テーブル・ビューワ（JupyterLab用）

このノートブックは、`nf_auto_runs/analysis/` に出力された **統合Excel**（`model_analysis_*.xlsx`）や、
**個別CSV**（`model_profile_*.csv` など）を自動検出して読み込み、JupyterLab 上で閲覧・検索・簡易集計できるようにします。

- まずは **パス設定** セルの `SEARCH_DIRS` に、実際の分析ファイル（Excel/CSV）があるフォルダを指定してください（複数指定可）。
- 統合Excelがあれば優先して読み込み、足りない表はCSVから補完します。
- `ipywidgets` がインストール済みなら、ドロップダウンで表を切り替えて検索・抽出できます。
- `ipywidgets` が無い場合は、フォールバックとして静的プレビューを表示します。

> 想定ファイル名の例：  
> `model_analysis_20251111_161847.xlsx`  
> `model_profile_20251111_161847.csv`, `dataset_profile_*.csv`, `training_state_*.csv`,  
> `weight_statistics_*.csv`, `model_complexity_*.csv`, `model_diagnosis_*.csv`,  
> `parameter_sensitivity_*.csv`, `optimization_suggestions_*.csv`


In [1]:

from pathlib import Path

# === パス設定（必要に応じて編集）==============================================
# 分析ファイル（Excel/CSV）が存在する候補ディレクトリをリストに入れてください。
SEARCH_DIRS = [
    Path.cwd() / "nf_auto_runs" / "analysis",
    Path.cwd(),                       # 現在の作業ディレクトリ直下
    Path.home() / "Downloads",        # ダウンロードフォルダ（必要なら）
]

# 追加で明示的に指定したいパスがあれば↓を編集
# SEARCH_DIRS.append(Path(r"C:\Users\YOUR_NAME\path\to\nf_auto_runs\analysis"))
# ============================================================================

# 検索対象のファイル名パターン
EXCEL_PATTERN = "model_analysis_*.xlsx"
CSV_PATTERNS = {
    "model_profile": "*model_profile_*.csv",
    "dataset_profile": "*dataset_profile_*.csv",
    "training_state": "*training_state_*.csv",
    "weight_statistics": "*weight_statistics_*.csv",
    "model_complexity": "*model_complexity_*.csv",
    "parameter_sensitivity": "*parameter_sensitivity_*.csv",
    "model_diagnosis": "*model_diagnosis_*.csv",
    "optimization_suggestions": "*optimization_suggestions_*.csv",
}


In [2]:

import pandas as pd
from typing import Dict, List, Optional
import re

def glob_all(pattern: str, roots: List[Path]) -> List[Path]:
    hits: List[Path] = []
    for r in roots:
        if r.exists():
            hits.extend(r.rglob(pattern))
    # 重複除去（解決後の絶対パスで比較）
    uniq = []
    seen = set()
    for p in hits:
        try:
            key = p.resolve()
        except Exception:
            key = p
        if key not in seen:
            seen.add(key)
            uniq.append(p)
    return uniq

def find_latest(paths: List[Path]) -> Optional[Path]:
    paths = [p for p in paths if p.exists()]
    if not paths:
        return None
    return sorted(paths, key=lambda p: p.stat().st_mtime, reverse=True)[0]

def try_read_csv(path: Path) -> pd.DataFrame:
    # 文字コードを総当たり（日本語CSV対策）
    encodings = ["utf-8", "utf-8-sig", "cp932", "latin-1"]
    last_err = None
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception as e:
            last_err = e
    raise last_err

def load_from_excel(xlsx_path: Path) -> Dict[str, pd.DataFrame]:
    dfs: Dict[str, pd.DataFrame] = {}
    try:
        xls = pd.ExcelFile(xlsx_path)  # openpyxlが必要
    except Exception as e:
        print(f"[info] Excel読み込みに失敗: {e}\n-> 'pip install openpyxl' を試してください。")
        return dfs
    for sheet in xls.sheet_names:
        try:
            df = pd.read_excel(xls, sheet_name=sheet)
            key = sheet.strip().lower()
            key = re.sub(r"\s+", "_", key)
            dfs[key] = df
        except Exception as e:
            print(f"[warn] シート '{sheet}' 読み込みエラー: {e}")
    return dfs

def load_from_csvs(roots: List[Path]) -> Dict[str, pd.DataFrame]:
    dfs: Dict[str, pd.DataFrame] = {}
    for key, pattern in CSV_PATTERNS.items():
        cands = glob_all(pattern, roots)
        if not cands:
            continue
        latest = find_latest(cands)
        if latest is None:
            continue
        try:
            df = try_read_csv(latest)
            dfs[key] = df
        except Exception as e:
            print(f"[warn] {key} CSV読込失敗 ({latest}): {e}")
    return dfs

def memory_usage_mb(df: pd.DataFrame) -> float:
    return float(df.memory_usage(index=True, deep=True).sum() / (1024 ** 2))


In [3]:

# 統合Excel → CSVの順で読み込み（Excelがあれば優先）
excel_files = glob_all(EXCEL_PATTERN, SEARCH_DIRS)
latest_excel = find_latest(excel_files)

tables: Dict[str, pd.DataFrame] = {}
if latest_excel:
    print(f"[info] 統合Excelを検出: {latest_excel}")
    tables = load_from_excel(latest_excel)

# CSVで補完（あるいはExcelが無い場合はCSVのみ）
csv_tables = load_from_csvs(SEARCH_DIRS)
# Excel側を優先しつつ、存在しないキーはCSVで補完
for k, v in csv_tables.items():
    if k not in tables:
        tables[k] = v

if not tables:
    print("""[not found] 分析ファイルが見つかりません。
以下のいずれかを配置してください：
- 統合Excel: model_analysis_YYYYMMDD_HHMMSS.xlsx
- 個別CSV: model_profile_*.csv, dataset_profile_*.csv, training_state_*.csv,
            weight_statistics_*.csv, model_complexity_*.csv, model_diagnosis_*.csv,
            parameter_sensitivity_*.csv, optimization_suggestions_*.csv
    """.strip())
else:
    print("=== 読み込んだテーブル一覧 ===")
    for key, df in tables.items():
        print(f"- {key}: {len(df)} 行 × {len(df.columns)} 列, 約 {memory_usage_mb(df):.2f} MB")


[info] 統合Excelを検出: c:\Users\hashimoto.ryohei\Downloads\zip\sonnet\nf_auto_runs\analysis\model_analysis_20251111_161847.xlsx
=== 読み込んだテーブル一覧 ===
- model_profile: 1 行 × 9 列, 約 0.00 MB
- dataset_profile: 1 行 × 11 列, 約 0.00 MB
- training_state: 1 行 × 7 列, 約 0.00 MB
- weight_statistics: 26 行 × 13 列, 約 0.01 MB
- model_complexity: 1 行 × 6 列, 約 0.00 MB
- model_diagnosis: 1 行 × 5 列, 約 0.00 MB
- optimization_suggestions: 1 行 × 7 列, 約 0.00 MB


In [ ]:

# 対話ビュー（ipywidgets）。未インストールでもこのセルは無害です。
try:
    import ipywidgets as W
    from IPython.display import display, clear_output

    if tables:
        table_names = sorted(tables.keys())

        dd_table = W.Dropdown(options=table_names, description='Table:', layout=W.Layout(width='300px'))
        txt_query = W.Text(value='', description='検索:', placeholder='部分一致（列名・値）')
        ms_cols   = W.SelectMultiple(options=[], description='列フィルタ:', layout=W.Layout(height='140px', width='300px'))
        num_head  = W.IntSlider(value=50, min=5, max=1000, step=5, description='先頭N行:')
        btn_show  = W.Button(description='表示', button_style='primary')
        out       = W.Output()

        def refresh_columns(*args):
            df = tables.get(dd_table.value)
            if df is not None:
                ms_cols.options = list(df.columns)
        dd_table.observe(refresh_columns, names='value')
        refresh_columns()

        def search_mask(df: pd.DataFrame, q: str):
            if not q:
                return pd.Series([True]*len(df), index=df.index)
            q = q.strip().lower()
            # 列名一致 or 値の部分一致（文字列化）
            col_hit = [c for c in df.columns if q in c.lower()]
            mask_val = df.apply(lambda col: col.astype(str).str.lower().str.contains(q, na=False) if col.dtype != 'object' else col.astype(str).str.lower().str.contains(q, na=False))
            row_mask = mask_val.any(axis=1)
            if col_hit:
                return row_mask | pd.Series([True]*len(df), index=df.index)  # 列名ヒットなら全行表示
            return row_mask

        def on_show_clicked(b):
            with out:
                clear_output()
                df = tables.get(dd_table.value)
                if df is None:
                    print("テーブルが見つかりません。")
                    return

                # 列フィルタ
                cols = list(ms_cols.value) if ms_cols.value else list(df.columns)
                df2 = df.loc[:, cols]

                # 検索
                q = txt_query.value
                mask = search_mask(df2, q)
                dfv = df2[mask].head(int(num_head.value))

                print(f"[info] {dd_table.value}: {len(df2)} 行 × {len(df2.columns)} 列 -> フィルタ後 {len(dfv)} 行を表示")
                display(dfv)

                # 列ごとの基本統計（数値列のみ）
                num_cols = df2.select_dtypes(include=['number']).columns.tolist()
                if num_cols:
                    print("\n[stats] 数値列の要約（head() の範囲とは独立）")
                    display(df2[num_cols].describe())

        btn_show.on_click(on_show_clicked)

        ui = W.VBox([
            W.HBox([dd_table, num_head]),
            W.HBox([txt_query, ms_cols]),
            btn_show,
            out
        ])
        display(ui)
    else:
        print("ロード済みテーブルが無いため、対話ビューはスキップします。")

except Exception as e:
    print(f"[info] ipywidgets が利用できないため、対話ビューをスキップします: {e}\n"
          f"-> 'pip install ipywidgets' 実行後、JupyterLab を再読み込みしてください。")



## 非対話プレビュー（必要に応じて実行）

以下のユーティリティで、対話ビューなしでもテーブルの概要を確認できます。


In [7]:

import pandas as pd

def show_table(name: str, head_n: int = 20):
    df = tables.get(name)
    if df is None:
        print(f"'{name}' は読み込まれていません。")
        return
    print(f"=== {name} ===")
    print(f"{len(df)} 行 × {len(df.columns)} 列, 約 {memory_usage_mb(df):.2f} MB")
    display(df.head(head_n))
    print("\n[info()]")
    print(df.info())

def show_value_counts(name: str, column: str, top_n: int = 20):
    df = tables.get(name)
    if df is None:
        print(f"'{name}' は読み込まれていません。")
        return
    if column not in df.columns:
        print(f"列 '{column}' は存在しません。候補: {list(df.columns)[:10]} ...")
        return
    vc = df[column].astype(str).value_counts().head(top_n)
    display(vc)

# 例：
show_table('model_profile', head_n=10)
show_value_counts('weight_statistics', 'layer_name', top_n=30)


=== model_profile ===
1 行 × 9 列, 約 0.00 MB


,model_dir_hash,model_alias,model_class,h,input_size,freq,total_params,trainable_params,hyperparameters
0,fde700f070a72ac01e0ecf3df9a2b6946e929f6b92b8ba...,dir(data_long)_parent_dir(N_features)_loto(num...,autotcn,NaN,NaN,NaN,0,0,{}



[info()]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   model_dir_hash    1 non-null      object 
 1   model_alias       1 non-null      object 
 2   model_class       1 non-null      object 
 3   h                 0 non-null      float64
 4   input_size        0 non-null      float64
 5   freq              0 non-null      float64
 6   total_params      1 non-null      int64  
 7   trainable_params  1 non-null      int64  
 8   hyperparameters   1 non-null      object 
dtypes: float64(3), int64(2), object(4)
memory usage: 204.0+ bytes
None


layer_name
hist_encoder.tcn.0.conv.weight            1
hist_encoder.tcn.0.conv.bias              1
hist_encoder.tcn.0.causalconv.0.weight    1
hist_encoder.tcn.0.causalconv.0.bias      1
hist_encoder.tcn.1.conv.weight            1
hist_encoder.tcn.1.conv.bias              1
hist_encoder.tcn.1.causalconv.0.weight    1
hist_encoder.tcn.1.causalconv.0.bias      1
hist_encoder.tcn.2.conv.weight            1
hist_encoder.tcn.2.conv.bias              1
hist_encoder.tcn.2.causalconv.0.weight    1
hist_encoder.tcn.2.causalconv.0.bias      1
hist_encoder.tcn.3.conv.weight            1
hist_encoder.tcn.3.conv.bias              1
hist_encoder.tcn.3.causalconv.0.weight    1
hist_encoder.tcn.3.causalconv.0.bias      1
hist_encoder.tcn.4.conv.weight            1
hist_encoder.tcn.4.conv.bias              1
hist_encoder.tcn.4.causalconv.0.weight    1
hist_encoder.tcn.4.causalconv.0.bias      1
context_adapter.weight                    1
context_adapter.bias                      1
mlp_decoder.layers.0.